<a href="https://colab.research.google.com/github/mjgb1988/Mari_prueba_cursoR/blob/master/TAREA_MARIA_JOSE_GONZALEZ_BERNAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TAREA: PROCESADO Y ANÁLISIS DE SERIES TEMPORALES DE IMÁGENES**

*A partir de la delimitación de tres parcelas de girasol y dos parcelas de trigo de la cuál se conoce la fecha de cosecha, y empleando los datos recogidos por la plataforma Sentinel 2 del programa de observación de la Tierra Copernicus, se desea obtener:*

*Cada una de las curvas de evolución de los apartados 1 y 2 se obtendrán atendiendo a los siguientes criterios:*

*a. No se tendrá en cuenta el porcentaje de nubes a la hora de crear     la colección de imágenes.*

*b. Atendiendo al criterio anterior, aplicar una máscara de nubes.*

*c. Establecer un porcentaje de nubes a la hora de crear la colección,   seleccionado por el alumno y generar una nueva gráfica de
   evolución.*

*d. A la colección del apartado "c" aplicar además una máscara de
   nubes.*


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=-r0g7k6BhwtgKa64vcrGghb5cZ5UFBmFq175LpUKwDc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xgHoiyjyO2M4HklrZPnOLUEcZDmnyE0tuyANAFGRYKI2CRjLDGtDGgM

Successfully saved authorization token.


**1. La curva de evolución de tres índices de vegetación, a elegir por el alumno, para cada cultivo.**

In [4]:
#Librerias que necesito para cargar un shapefile desde Google Drive.
!pip install geetools 
!pip install pycrs 

     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 225kB 8.6MB/s 
  Created wheel for geetools: filename=geetools-0.4.15-cp36-none-any.whl size=98234 sha256=9a85d3e2e097bdf4fd3a82a28770adbad7983a849b811e798bb0e3d3d54b66fa
  Stored in directory: /root/.cache/pip/wheels/a9/29/4b/8b23d7a814fd5b03acab81a0a721c4dc0d782fed28760b636a
  Created wheel for pyshp: filename=pyshp-2.1.0-cp36-none-any.whl size=32609 sha256=26fe941e939bfd6b493086f5ad5c64715b4a3aeace67a44a563ec5d7bab6603f
  Stored in directory: /root/.cache/pip/wheels/a6/0c/de/321b5192ad416b328975a2f0385f72c64db4656501eba7cc1a
Successfully built geetools pyshp
  Created wheel for pycrs: filename=PyCRS-1.0.2-cp36-none-any.whl size=32683 sha256=41dd432f022a623b81020e2d5abeb60b27ca3c312f41bef4e05bbe686e9bae9e
  Stored in directory: /root/.cache/pip/wheels/1a/c9/56/648f762bd41fdb03454be51b8a7069795925168833cb61f316
Successfully built pycrs


In [10]:
#Acceso a nuestro espacio en Google Drive.
from google.colab import auth, drive
auth.authenticate_user()

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Función para definir coordenadas centrales de colección de imágenes.
def centro_colección(coleccion):
  lista=colección.toList(coleccion.size())
  latitud_suma=0
  longitud_suma=0
  for i in range(0,lista.size().getInfo()):
    img=ee.Image(lista.get(i))
    coordenadas=imagen.get('system:footprint').getInfo()['coordinates']
    longitud=0
    latitud=0
    for j in range(len(coordenadas)):
      longitud= longitud+coordenadas[j-1][0]
      latitud=latitud+coordenadas[j-1][1]

      longitud=longitud/(len(coordenadas))
      latitud=latitud/(len(coordenadas))

      latitud_suma=latitud_suma+latitud
      longitud_suma=longitud_suma+longitud

  latitud_central=latitud_suma/lista.size().getInfo()
  longitud_central=longitud_suma/lista.size().getInfo()

  return (latitud_central,longitud_central)

                 

In [0]:
#Función Mapdisplay.

import folium

EE_TITLES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center:Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bricht,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center, tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr = 'Google Earth Engine',
            overlay = True,
            name = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz


In [32]:
def NDVI(entrada):
  ndvi = entrada.normalizedDifference(['B8','B4'])
  return (ndvi)

coleccion_sentinel = ee.ImageCollection('COPERNICUS/S2_SR')\
           .filterBounds(parcela)\
           .filterDate('2019-01-01','2020-02-14')\
           .filterMetadata ('CLOUD_COVERAGE_ASSESMENT','less_than',5)

coleccion_ndvi=coleccion_sentinel.map(NDVI)

num_imagenes = coleccion_sentinel.size()
print('Número de imágenes colección Sentinel 2: ', num_imagenes.getInfo())
print('Número de imágenes colección NDVI: ', coleccion_ndvi.size().getInfo())
print ()
print('Información de una de las imágenes de la colección de NDVI')
imagen = ee.Image(coleccion_ndvi.first());
imagen.getInfo()


Número de imágenes colección Sentinel 2:  0
Número de imágenes colección NDVI:  0

Información de una de las imágenes de la colección de NDVI


In [29]:
import os
from geetools import batch

#Creo una variable "path_file" que contiene la ruta del archivo shapefile.
#Creo otra variable "parcela" que lee el contenido de "path_file".
path_file = os.path.join('/content/drive/My Drive/parcelas/parcelas.shp')
parcela = batch.Import.table.fromShapefile(path_file)

coleccion = ee.ImageCollection('COPERNICUS/S2')\
               .filterBounds(parcela)\
               .select(['B8', 'B4', 'B3'])\
               .filterDate ('2019-01-01','2020-02-14')\

mosaico = coleccion.mosaic()

num_imagenes = coleccion.size()
print('Número de imágenes: ', num_imagenes.getInfo())

#Defino los parámetros de visualización.
vis_params = {
    'min': 500,
    'max': 2500,
    'bands': ['B8', 'B3', 'B4'],
}

#Determino el centroide de las geometrias de la tabla.
centroide = parcela.geometry().centroid().getInfo()['coordinates']
centroide.reverse()

#Creo un diccionario con la colección de imáganes y las parcelas.
dicc = {
    'coleccion': coleccion.getMapId(vis_params),
    'mosaico': mosaico.clip(parcela).getMapId(vis_params),
    'parcela':parcela.getInfo()
}

#Represento resultados.
Mapdisplay(centroide, dicc, zoom_start= 15)

Número de imágenes:  328


**2. La curva de evolución a nivel individual de cada una de las parcelas facilitadas de los índices anteriores.**

**3. Buscar en los dataset que ofrece Google Earth Engine para pintar en una gráfica la evolución de, al menos, una variable climática coincidente temporalmente con el ciclo de los cultivos de trabajo.**